# Library

In [ ]:
!pip install unsloth "xformers==0.0.28.post2"

In [ ]:
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import (
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from unsloth import is_bfloat16_supported
from unsloth import FastLanguageModel
from unsloth.chat_templates import (
    get_chat_template,
    train_on_responses_only,
    standardize_sharegpt,
)

# Unsloth

models: https://huggingface.co/unsloth

LLM(대규모 언어 모델)의 훈련을 최적화하기 위한 도구. <br>
훈련 속도를 높이고 메모리 사용량을 줄이며 정확성을 유지. <br>

<br>

<font style="font-size:20px"> 특징 </font>

1. 기존에 비해 LLM 훈련 속도를 약 30배 향상시킬 수 있음 <br>
ex) Alpaca 모델의 훈련 시간이 85시간이 걸렸는데 비해, Unsloth의 경우 3시간으로 단축.
2. 메모리 사용량 감소 <br>
최적화된 메모리 관리 기술을 사용하여 훈련 중에 메모리 양을 줄일 수 있음 <br>
-> 배치 사이즈를 키울 수 있어 더 빠른 훈련 모델을 훈련할 수 있게 함
3. 정확성 유지
Unsloth는 훈련 속도를 높이면서도 모델의 정확도 유지 <br>
4. 다양한 하드웨어 지원 <br>
NVIDIA, Intel 및 AMD GPU를 지원하여 다양한 하드웨어 환경에서 사용 가능
5. LoRA 지원 <br>
Unsloth를 사용하면 LoRA를 훨씬 쉽고 빠르게 이용 가능

<br>

<font style="font-size:20px"> 적용 기술 </font>

1. 커널 최적화 기술 <br>
Unsloth는 Triton과 같은 커널 최적화 기술을 사용하여 모델의 핵심 연산 최적화. <br>
\- Triton 커널은 PyTorch 모델의 연산을 GPU에서 더 효율적으로 실현할 수 있도록 최적화된 커널로, 일반적인 PyTorch 연산보다 훨씬 빠른 속도 제공. <br>
2. 메모리 최적화 <br>
Unsloth는 메모리 사용을 최적화하여 GPU의 메모리 사용량을 줄임 <br>
-> 처리할 수 있는 데이터 양을 늘려 더 많은 연산을 동시에 수행할 수 있게 함
3. 경사하강법 최적화 <br>
Unsloth는 경사하강법 최적화 과정을 최적화하여 모델의 학습 속도를 높임 <br>
-> 모델의 가중치를 더 빠르게 업데이트하여 학습 속도를 높이는 데 도움이 됨
4. 병렬화 및 배치 처리 <br>
Unsloth는 병렬화 기술과 배치 처리를 통해 모델의 학습을 더 효율적으로 수행 <br>
-> 더 큰 사이즈의 데이터를 처리할 수 있게 하고 배치 학습 효율을 높임

## 사용 방법

> ```python
> # model & tokenizer load
> model_name = 'unsloth/Llama-3.2-3B-Instruct'
> model, tokenizer = FastLanguageModel.from_pretrained(
>     model_name = model_name,
>     load_in_4bit=True,
>     dtype=None, # auto detection
>     max_seq_length=2048,    # 아무 값이나 선택해도 무방
> )
> 
> model = FastLanguageModel.get_peft_model(
>     model,
>     r=8, # Suggested 8, 16, 32, 64, 128
>     target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj',
>                     'gate_proj', 'up_proj', 'down_proj',],
>     lora_alpha=16,
>     lora_dropout = 0,
>     bias="none",
>     use_gradient_checkpointing="unsloth",
>     random_state=0,
> )
> 
> # load chat template tokenizer
> tokenizer = get_chat_template(
>     tokenizer,
>     chat_template='llama-3.1',
> )
> 
> # preprocessing function
> def formatting_prompts_func(examples):
>     conversations = examples['conversations']
>     texts = [
>         tokenizer.apply_chat_template(
>             conversation,
>             tokenize=False,
>             add_generation_prompt=False)
>         for conversation
>         in conversations
>     ]
> 
>     return {
>         'text' : texts,
>     }
> 
> # data load and preprocessing
> dataset = load_dataset(<dataset>)
> dataset = standardize_sharegpt(dataset)
> dataset = dataset.map(formatting_prompts_func, batched=True)
> 
> # training_args
> training_args = TrainingArguments(
>         per_device_train_batch_size=2,
>         gradient_accumulation_steps=4,
>         warmup_steps=5,
>         # num_train_epochs=1,
>         max_steps=60,
>         learning_rate=2e-4,
>         fp16=not is_bfloat16_supported(),   # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
>         bf16=is_bfloat16_supported(),       # bf16 사용 여부, bf16이 지원되는 경우에만 사용
>         logging_steps=1,
>         optim='adamw_8bit',
>         weight_decay=0.01,
>         lr_scheduler_type='linear',         # 스케줄러 유형
>         seed=0,
>         output_dir=<dir>,
>         report_to="WandB",
> )
> 
> # trainer
> trainer = SFTTrainer(
>     model=model,
>     tokenizer=tokenizer,
>     train_dataset=dataset,
>     dataset_text_field='text',
>     data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),
>     dataset_num_proc=16,
>     packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
>     args=training_args
> )
> 
> # trainer setting
> trainer = train_on_responses_only(
>     trainer,
>     instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
>     response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
> )
> 
> # train
> trainer_stats = trainer.train()
> ```

In [ ]:
# 사용할 모델 이름 설정
model_name = 'unsloth/Llama-3.2-3B-Instruct'

# FastLanguageModel을 통해 모델과 토크나이저를 사전 학습된 모델에서 불러옴
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,          # 불러올 모델 이름
    load_in_4bit=True,              # 4비트 양자화를 사용하여 메모리 효율성을 높임
    dtype=None,                     # 데이터 타입을 지정하지 않아 기본 타입을 사용
    max_seq_length=2048,            # 최대 시퀀스 길이를 2048로 설정
)

# LoRA (Low-Rank Adaptation) 기반의 미세 조정 기법을 적용하여 모델 업데이트
model = FastLanguageModel.get_peft_model(
    model,
    r=8,                            # LoRA의 랭크(r) 값 설정
    target_modules=[                # LoRA를 적용할 모델의 특정 모듈을 지정
        'q_proj', 'k_proj', 'v_proj', 'o_proj',
        'gate_proj', 'up_proj', 'down_proj',
    ],
    lora_alpha=16,                  # LoRA의 알파(alpha) 값 설정
    lora_dropout=0,                 # 드롭아웃 비율을 0으로 설정하여 드롭아웃 비활성화
    bias='none',                    # 바이어스 추가 설정
    use_gradient_checkpointing='unsloth', # 메모리 최적화를 위해 gradient checkpointing 사용
    random_state=0,                 # 시드 값을 설정해 재현성 확보
)

# 토크나이저에 대해 사용자 정의 채팅 템플릿을 적용하여 대화 형식을 맞춤화
tokenizer = get_chat_template(
    tokenizer,
    chat_template='llama-3.1',      # 채팅 템플릿으로 'llama-3.1' 형식 사용
)

In [ ]:
# 데이터셋의 각 샘플을 전처리하는 함수 정의
def preprocessing(sample):
    # 샘플 내의 대화(conversations) 필드를 가져옴
    conversations = sample.get('conversations')
    
    # 각 conversation을 템플릿에 맞춰 텍스트 형식으로 변환
    texts = [
        tokenizer.apply_chat_template(
            conversation,           # 대화 내용
            tokenize=False,         # 토큰화 생략 (토큰화는 나중에 진행)
            add_generation_prompt=False # 생성 프롬프트 추가 안 함
        ) for conversation in conversations
    ]

    # 변환된 텍스트 리스트를 딕셔너리 형태로 반환
    return {'text': texts}

# FineTome-100k 데이터셋 로드, 학습(train) 데이터셋 분할 사용
dataset = load_dataset('mlabonne/FineTome-100k', split='train')

# 대화 형식을 표준화 (ShareGPT와 유사한 형식으로 변환)
dataset = standardize_sharegpt(dataset)

# 데이터셋을 맵핑하여 각 샘플에 대해 전처리 수행
dataset = dataset.map(preprocessing, batched=True)

In [ ]:
# TrainingArguments 클래스를 사용해 학습에 필요한 다양한 하이퍼파라미터를 설정
training_args = TrainingArguments(
    per_device_train_batch_size=2,          # 각 디바이스(GPU)에서의 배치 사이즈를 2로 설정
    gradient_accumulation_steps=4,          # 그라디언트 누적 횟수를 4로 설정하여 더 큰 배치 효과 달성
    warmup_steps=10,                        # 학습 초기에 워밍업 단계로 사용할 스텝 수
    max_steps=100,                          # 전체 학습 스텝 수 제한
    learning_rate=2e-4,                     # 학습률을 0.0002로 설정
    fp16=not is_bfloat16_supported(),       # FP16(half precision) 사용 여부 설정; bfloat16을 지원하지 않을 때 사용
    bf16=is_bfloat16_supported(),           # BF16(bfloat16)을 지원할 경우 사용 여부 설정
    logging_steps=10,                       # 학습 로그 기록 간격을 10 스텝마다로 설정
    seed=0,                                 # 시드 값 설정으로 재현성 확보
    output_dir='./llama3.2_w_unsloth',      # 학습 결과와 체크포인트를 저장할 디렉터리
    report_to='wandb',                      # 로그 및 메트릭을 `wandb`에 보고
)

# SFTTrainer 인스턴스를 생성하여 모델 학습 준비
trainer = SFTTrainer(
    model=model,                            # 학습할 모델
    tokenizer=tokenizer,                    # 모델의 토크나이저
    train_dataset=dataset,                  # 학습에 사용할 데이터셋
    dataset_text_field='text',              # 데이터셋의 텍스트 필드명
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer), # 데이터 수집 및 전처리 설정
    dataset_num_proc=16,                    # 데이터셋 처리에 사용할 프로세스 수
    packing=False,                          # 패킹 설정 (여기서는 패킹 사용 안 함)
    args=training_args,                     # 학습에 사용할 TrainingArguments 설정
)

# 학습 데이터를 사용자 질문과 응답 데이터만으로 제한하는 함수 호출
trainer = train_on_responses_only(
    trainer,
    instruction_part='<|start_header_id|>user<|end_header_id|>\n\n',   # 사용자 입력의 시작 및 종료 부분
    response_part='<|start_header_id|>assistant<|end_header_id|>\n\n', # 어시스턴트 응답의 시작 및 종료 부분
)

# 모델 학습 시작
results = trainer.train()